# Exercise 3: Numerical Instability and Stability Analysis

## Learning Objectives

By completing this exercise, you will:
- Recognize symptoms of numerical instability
- Understand the relationship between finite difference direction and wave propagation
- Investigate the CFL stability condition empirically
- Implement backward and centered finite difference schemes
- (Optional) Solve the nonlinear Burgers' equation

---

## Introduction

Numerical instability is one of the critical problems in computational physics. In this exercise, we will see how **violent** numerical instabilities can be and learn to recognize their symptoms.

---

## Part 1: Numerical Instability — Violent Development

### Task 1.1: Trigger Instability by Changing Sign of $a$

Repeat the numerical simulation from [Exercise 2b](https://github.com/AST-Course/AST5110/blob/main/notebooks/ex_2b.ipynb), but now with $a = +1$ (instead of $a = -1$).

**Steps:**
1. Use a moderate number of intervals (e.g., 128)
2. Observe what happens
3. Count after how many timesteps instability becomes evident

**Follow-up:** Repeat with 1024 points (and other powers of 2). Is the experiment stable now? After how many timesteps does instability appear?

### Task 1.2: Implement Backward Finite Differences

For $a = 1$, use **backward** finite differencing for the spatial derivative:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_i-u_{i-1}}{\Delta x}  \tag{1}$$

**Implementation:** Fill in the function `deriv_bck` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_3.py). Use the `lambda` function `ddx` and select the proper limits for `bnd_limits`.

> **Important:** Take care to also change the implementation of the boundary condition!

**Question:** Does the unstable character of the code change with backward differencing?

For $a = -1$, use **backward** finite differencing for the spatial derivative, Is unstable?



---

## Part 2: Centered Differences

### Task 2.1: Implement Centered Differences

Use **centered finite differences** for the spatial derivative:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i-1}}{2\Delta x}   \tag{2}$$

If not already done, fill in the function `deriv_cent` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_1.py). Use the `lambda` function `ddx` and select proper limits for `bnd_limits`. Set `cfl_cut = 0.3`.

**Questions:**
1. Is any instability apparent with centered differences?
2. Does the situation change when you change the sign of $a$?

> **Note on Boundary Conditions:** Define `xx` so that:
> - The endpoint $x = x_0$ coincides with `xx[1]` (second element)
> - The endpoint $x = x_f$ coincides with `xx[nump-1]` (last element)
> 
> For the boundary condition on array `uun` at time $t + \Delta t$:
> ```python
> uun[0] = uun[nump-2]
> uun[nump-1] = uun[1]
> ```

---

## Part 3: Stability of Non-Centered Schemes

### The CFL Condition

In the previous exercises, we saw that for $a > 0$, a **backward-oriented** finite-difference scheme yields stability. However, we used a specific value $\Delta t = 0.98 \, \Delta x / |a|$. 

**Question:** Can $\Delta t$ be chosen arbitrarily large?



### Task 3.1: Investigate the CFL Threshold

Run the program with different values of `cfl_cut`:


| Test | $\Delta t$ | `cfl_cut` |
|------|-----------|-----------|
| 1 | $0.5 \, \Delta x / a$ | 0.5 |
| 2 | $0.99 \, \Delta x / a$ | 0.99 |
| 3 | $1.01 \, \Delta x / a$ | 1.01 |
| 4 | $2.0 \, \Delta x / a$ | 2.0 |

**Questions:**
1. Do these values maintain good stability properties?
2. Does there appear to be a **threshold** in $\Delta t$ for instability?


---

## Part 4: Burgers' Equation (Optional)

> **Note:** Parts 4A and 4B below are independent optional extensions. You may complete either or both.

### Part 4A: The Nonlinear Advection Equation

Consider **Burgers' equation**:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{3}$$

This can be viewed as the advection equation with $a(x,t,u) = u$.

**Physical significance:** This equation describes:
- Motion of a non-accelerated fluid with arbitrary initial velocity
- Spontaneous formation of **discontinuities** (shocks)


### Discussion: Shock Formation

In the numerical solution, we see the initial condition evolving into something resembling a **discontinuity**. This happens because:

- Characteristic curves $dx_p/dt = a(x,t,u) = u$ are steeper (faster) for larger $u$
- The solution is constant along characteristics
- Faster elements catch up to slower ones ahead, causing **steepening**

**Numerical considerations:**

1. **Accuracy:** Computing large derivatives across the jump cannot be very accurate, yet the program continues without crashing

2. **Weak solutions:** When discontinuities form, one must consider **jump relations** (internal boundary conditions) across the discontinuity

3. **Open question:** Does our simple numerical scheme approximate the mathematically-exact weak solution?

> **Note:** The term $u \, \partial u/\partial x$ appears in fluid dynamics acceleration. Much of the beautiful physics in the universe is due to such nonlinearities!

---

### Part 4B: Forward Derivative and Lax Method

> **Prerequisite:** Complete [Exercise 4](https://github.com/AST-Course/AST5110/blob/main/notebooks/ex_4.ipynb) first.

### Task 4A.1: Implement Burgers' Equation

Modify your program to solve eq. (3) by setting $a(x_i, t^n) = u^n_i$.

**Setup:**
- Domain: $x \in [-1.4, 2.0]$
- Initial condition:
$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right) \right] + B \tag{5}$$
- Parameters: $A = 1.0$, $x_c = 0.70$, $W = 0.1$, $B = 0.3$
- Final time: $t_f = 100$

**Implementation:** Fill in functions `evolv_uadv_burgers` and `step_uadv_burgers` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_3.py).

**Questions:**
1. Explain in physical/mathematical terms the solution you observe
2. Change to $A = -0.02$ and explain the result

### Task 4B.1: Compare Methods for Burgers' Equation

Using the same setup as Part 4A, solve using:

1. **Forward spatial derivative:**
$$u^{n+1}_j = u^{n}_j - \frac{\Delta t}{\Delta x} u^{n}_j(u^{n}_j - u^{n}_{j+1})$$

2. **Lax method** (from [exercise 4](https://github.com/AST-Course/AST5110/blob/main/notebooks/ex_4.ipynb))

**Questions:**
1. What do you observe?
2. Why do you think the solution might not be correct?

> **Hint:** The issue relates to **flux conservation**. When discretizing $u \partial u/\partial x$, different formulations (e.g., $u_j (u_{j+1} - u_j)/\Delta x$ vs. $(u_{j+1}^2 - u_j^2)/(2\Delta x)$) are not equivalent at the discrete level. The latter preserves the integral of $u^2$ (energy), while the former does not. See [Staggered Mesh](https://github.com/AST-Course/AST5110/wiki/Staggered-mesh) for details on conservative formulations.

### Further Reading

See [Staggered Mesh](https://github.com/AST-Course/AST5110/wiki/Staggered-mesh) documentation on how a staggered mesh allows preserving conservation properties.

---

## Hints

<details>
<summary><b>Hint 1: Why does sign of $a$ matter?</b></summary>

The **upwind direction** matters! Information propagates in the direction of $a$:
- If $a > 0$: information comes from the left → use backward differences
- If $a < 0$: information comes from the right → use forward differences

Using the wrong direction means you're extrapolating rather than interpolating.
</details>

<details>
<summary><b>Hint 2: Recognizing instability</b></summary>

Signs of numerical instability:
- Rapid growth of oscillations (often starting small)
- Values growing without bound
- High-frequency "wiggles" appearing in the solution
- Solution blowing up to `inf` or `NaN`
</details>

<details>
<summary><b>Hint 3: The CFL condition physically</b></summary>

The CFL number $C = |a| \Delta t / \Delta x$ represents how many grid cells information travels per timestep. If $C > 1$, information "jumps over" grid points, leading to instability.
</details>

---